# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Axel Gallardo

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 00:50:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!apt-get update
!apt-get install -y netcat

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 566, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,750 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,584 kB]
Fetched 5,718 kB in 15s (393 kB/s)                                             
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
netcat is already the newest version (1.218-4ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [3]:
from pcamarillor.spark_utils import SparkUtils
from pyspark.sql.functions import to_timestamp, date_format, col

logs_schema = SparkUtils.generate_schema([
    ("Date","string"),("Level","string"),("Message","string"),("Module","string")
])

logs_df = (spark.readStream
    .format("json")
    .schema(logs_schema)
    .option("maxFilesPerTrigger", 1)               # 1 archivo por micro-batch
    .option("pathGlobFilter", "*.json")            # solo json
    .load("/opt/spark/work-dir/data/lab07/")       # <= MISMA ruta que el generador
)

result_df = (logs_df
    .withColumn("ts", to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss"))
    .select(
        date_format("ts","yyyy-MM-dd HH:mm:ss").alias("Date"),
        "Level","Message","Module"
    )
)

# Sinks: console + memory (para ver en la celda)
console_q = (result_df.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .option("numRows", 50)
    .trigger(processingTime="2 seconds")
    .start()
)

memory_q = (result_df.writeStream
    .outputMode("append")
    .format("memory")
    .queryName("lab07_view")
    .trigger(processingTime="2 seconds")
    .start()
)

print("Streams iniciados (carpeta limpia). Ahora lanza el generador.")

Streams iniciados (carpeta limpia). Ahora lanza el generador.


In [4]:
import sys
from threading import Thread
from time import sleep

# importa tu generador desde lib
sys.path.append("Main/O2025_ESI3914B/spark/notebooks/lib")
from axelgallardo.lab07_generator import generate_logs

# Lanza: crea 5 archivos, 8 líneas cada uno, separados 3s
Thread(target=generate_logs, kwargs=dict(n_files=5, lines_per_file=8, delay_between_files=3), daemon=True).start()

# Refresca para VER los micro-batches en la notebook
for _ in range(12):
    spark.sql("SELECT * FROM lab07_view").show(truncate=False)
    print(console_q.lastProgress)
    sleep(2)

console_q.awaitTermination(10)


+----+-----+-------+------+
|Date|Level|Message|Module|
+----+-----+-------+------+
+----+-----+-------+------+

{
  "id" : "96fee2b1-44b8-4b2b-983d-68266042a885",
  "runId" : "eab51994-747a-432b-ae08-208833cdaa34",
  "name" : null,
  "timestamp" : "2025-10-10T00:50:31.652Z",
  "batchId" : 0,
  "batchDuration" : 39,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "latestOffset" : 29,
    "triggerExecution" : 36
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/opt/spark/work-dir/data/lab07]",
    "startOffset" : null,
    "endOffset" : null,
    "latestOffset" : null,
    "numInputRows" : 0,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@42f4262a",
    "numOutputRows" : -1
  }
}
+----+-----+-------+------+
|Date|Level|Message|Module|
+----+-----+-------+------+
+----+--

[Stage 0:>                  (0 + 1) / 1][Stage 1:>                  (0 + 0) / 1]

{
  "id" : "96fee2b1-44b8-4b2b-983d-68266042a885",
  "runId" : "eab51994-747a-432b-ae08-208833cdaa34",
  "name" : null,
  "timestamp" : "2025-10-10T00:50:31.652Z",
  "batchId" : 0,
  "batchDuration" : 39,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "latestOffset" : 29,
    "triggerExecution" : 36
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/opt/spark/work-dir/data/lab07]",
    "startOffset" : null,
    "endOffset" : null,
    "latestOffset" : null,
    "numInputRows" : 0,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@42f4262a",
    "numOutputRows" : -1
  }
}
+----+-----+-------+------+
|Date|Level|Message|Module|
+----+-----+-------+------+
+----+-----+-------+------+

{
  "id" : "96fee2b1-44b8-4b2b-983d-68266042a885",
  "runId" : "eab51994-747a-432b-ae08-2088

-------------------------------------------
Batch: 0
-------------------------------------------
{
  "id" : "96fee2b1-44b8-4b2b-983d-68266042a885",
  "runId" : "eab51994-747a-432b-ae08-208833cdaa34",
  "name" : null,
  "timestamp" : "2025-10-10T00:50:31.652Z",
  "batchId" : 0,
  "batchDuration" : 39,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "latestOffset" : 29,
    "triggerExecution" : 36
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/opt/spark/work-dir/data/lab07]",
    "startOffset" : null,
    "endOffset" : null,
    "latestOffset" : null,
    "numInputRows" : 0,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@42f4262a",
    "numOutputRows" : -1
  }
}
+-------------------+-----+---------------------+-------------+
|Date               |Level|Message          

False